In [1]:
import numpy as np
import open3d as o3d
import os
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def load_landmarks(txt_path):
    """
    Load landmark coordinates exported from CloudCompare.
    Supports both CSV (x,y,z) and space-separated formats.
    Returns: (N,3) numpy array
    """
    pts = []
    with open(txt_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            if "," in line:
                vals = line.split(",")
            else:
                vals = line.split()

            if len(vals) < 3:
                continue

            try:
                x = float(vals[0])
                y = float(vals[1])
                z = float(vals[2])
                pts.append([x, y, z])
            except ValueError:
                continue

    return np.asarray(pts, dtype=np.float64)

def compute_rigid_transform(P, Q):
    """
    Given 2 list of points:
      P: (N, 3) moving object points (mov_landmarks)
      Q: (N, 3) reference object points (ref_landmarks)
    Solve Rigid transformation T so that Q ≈ R @ P + t
    Return:
     4x4 transform matrix T
    """
    assert P.shape == Q.shape
    # Centroid
    centroid_P = P.mean(axis=0)
    centroid_Q = Q.mean(axis=0)

    # Decentroid
    P_centered = P - centroid_P
    Q_centered = Q - centroid_Q

    # Covariance
    H = P_centered.T @ Q_centered

    # SVD
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # Reflection case
    if np.linalg.det(R) < 0:
        Vt[2, :] *= -1
        R = Vt.T @ U.T

    # translation
    t = centroid_Q - R @ centroid_P

    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T


In [3]:
print(os.getcwd())

/users/nfs/Etu0/21400500/PRAT


In [5]:
ref_pcd_path = "Data/ICP_test/ref_Tete_D.ply"
mov_pcd_path = "Data/ICP_test/move_Dragon_01_Transform.ply"
ref_lm_path  = "Data/ICP_test/picking_list_Tete_D.txt"
mov_lm_path  = "Data/ICP_test/picking_list_Dragon_01_Transform.txt"

# Read Point Cloud
ref_pcd = o3d.io.read_point_cloud(ref_pcd_path)
mov_pcd_raw = o3d.io.read_point_cloud(mov_pcd_path)

print(ref_pcd)
print(mov_pcd_raw)

# Read landmarks
Q = load_landmarks(ref_lm_path)  # reference
P = load_landmarks(mov_lm_path)  # moving

print("Landmark pairs:", P.shape)


PointCloud with 1349372 points.
PointCloud with 1711170 points.
Landmark pairs: (5, 3)


In [16]:
T_init = compute_rigid_transform(P, Q)
print("Initial transform (from landmarks):\n", T_init)
np.savetxt("T_landmark_4x4.txt", T_init)


# mov_pcd_landmark = mov_pcd.transform(T_init.copy())

# o3d.io.write_point_cloud("../Data/ICP_test/dragon_after_landmark.ply", mov_pcd_landmark)

mov_after_landmark = copy.deepcopy(mov_pcd_raw)
mov_after_landmark.transform(T_init)
o3d.io.write_point_cloud("../Data/ICP_test/dragon_after_landmark.ply", mov_after_landmark)



Initial transform (from landmarks):
 [[ 9.62467722e-01  2.70397248e-01 -2.32639644e-02 -3.62131506e+01]
 [-2.70754231e-01  9.50764689e-01 -1.50793408e-01 -8.51340723e+02]
 [-1.86555667e-02  1.51432605e-01  9.88291524e-01 -2.97968502e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


True

In [17]:
max_corr_dist = 10.0

result_icp = o3d.pipelines.registration.registration_icp(
    mov_pcd_raw,                   # source
    ref_pcd,                       # target
    max_corr_dist,
    T_init,                        # init_trans from Landmark coarse registration
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(
        max_iteration=100
    )
)

print("ICP result:")
print("  fitness:", result_icp.fitness)
print("  inlier_rmse:", result_icp.inlier_rmse)
print("  T_icp:\n", result_icp.transformation)


ICP result:
  fitness: 0.8167061133610337
  inlier_rmse: 1.7607851593499075
  T_icp:
 [[ 9.54561581e-01  2.96914653e-01 -2.55710273e-02 -4.80688633e+01]
 [-2.97020892e-01  9.40874457e-01 -1.62892129e-01 -9.31408116e+02]
 [-2.43059335e-02  1.63085697e-01  9.86312464e-01 -4.09150237e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [18]:
T_total = result_icp.transformation
np.savetxt("T_icp_total_4x4.txt", T_total)

mov_after_icp = copy.deepcopy(mov_pcd_raw)
mov_after_icp.transform(T_total)
o3d.io.write_point_cloud("../Data/ICP_test/dragon_after_icp_python.ply", mov_after_icp)


print("Saved aligned point cloud to dragon_after_icp_python.ply")
print("Saved ICP transform to T_icp_4x4.txt")


Saved aligned point cloud to dragon_after_icp_python.ply
Saved ICP transform to T_icp_4x4.txt


### Comparison
Apply icp to mov dragon

[11:24:13] [ComputeDistances] Mean distance = 11.4888 / std deviation = 26.862

Default software algo:

[11:58:09] [ComputeDistances] Mean distance = 11.5154 / std deviation = 26.9277

### FPFH + RANSAC

In [39]:
def preprocess_point_cloud(pcd: o3d.geometry.PointCloud, voxel_size):
    """
    Downsample + normal estimation + FPFH feature extraction.

    Returns:
        pcd_down: downsampled point cloud with normals
        fpfh:     o3d.pipelines.registration.Feature (shape: 33 x N)
    """
    # Voxel downsample 
    pcd_down = pcd.voxel_down_sample(voxel_size)

    # Remove statistical outliers
    # Considered as outlier if avg dist among neighbors >= global avg dist + std_ratio * std
    pcd_down, _ = pcd_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # Estimate normals
    radius_normal = voxel_size * 2.0
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
    )

    # Compute FPFH feature
    radius_feature = voxel_size * 5.0
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100)
    )

    return pcd_down, fpfh


def draw_registration_result(source, target, transformation):
    """Visualize alignment with two colors."""
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1.0, 0.0, 0.0])  # red
    target_temp.paint_uniform_color([0.0, 0.5, 1.0])  # blue
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


def global_registration_ransac(src_down, tgt_down, src_fpfh, tgt_fpfh, voxel_size: float):
    """
    RANSAC-based global registration on FPFH feature matches.
    """
    distance_threshold = voxel_size * 1.5

    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        src_down, tgt_down,
        src_fpfh, tgt_fpfh,
        mutual_filter=True, # ensure src/tar NN corespondance
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False), # rigid transform
        ransac_n=4, # sample point pairs to solve T 
        checkers=[ # pre filter
            # Enforce similar edge length ratios to reject degenerate matches
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            # Reject matches with too large geometric distance
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold),
        ],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(
            max_iteration=100000,  
            confidence=0.999
        )
    )
    return result


def refine_registration_icp(src_full, tgt_full, init_T, voxel_size, use_point_to_plane=False):
    """
    ICP refinement (point-to-plane is usually better if normals are reliable).
    """
    # Set a tighter threshold than RANSAC
    max_corr_dist = voxel_size * 1.0

    if use_point_to_plane:
        # Normals are required for point-to-plane ICP (target normals are used)
        tgt_full.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0, max_nn=30)
        )
        src_full.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0, max_nn=30)
        )
        estimation = o3d.pipelines.registration.TransformationEstimationPointToPlane()
    else:
        estimation = o3d.pipelines.registration.TransformationEstimationPointToPoint()

    result_icp = o3d.pipelines.registration.registration_icp(
        src_full, tgt_full,
        max_corr_dist,
        init_T,
        estimation_method=estimation,
        criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=80)
    )
    return result_icp



In [66]:

ref_path = "Data/ICP_test/ref_Tete_D.ply"
mov_path = "Data/ICP_test/move_Dragon_01_Transform.ply"

ref_pcd = o3d.io.read_point_cloud(ref_path)
mov_pcd = o3d.io.read_point_cloud(mov_path)

print("Loaded:")
print("  ref:", ref_pcd)
print("  mov:", mov_pcd)


voxel_size = 5.0

# 1) Preprocess for FPFH + RANSAC on downsampled point clouds

ref_down, ref_fpfh = preprocess_point_cloud(ref_pcd, voxel_size)
mov_down, mov_fpfh = preprocess_point_cloud(mov_pcd, voxel_size)

print("Downsampled:")
print("  ref_down:", ref_down)
print("  mov_down:", mov_down)
print("FPFH dims:", ref_fpfh.data.shape, mov_fpfh.data.shape)  # (33, N)



Loaded:
  ref: PointCloud with 1349372 points.
  mov: PointCloud with 1711170 points.
Downsampled:
  ref_down: PointCloud with 57450 points.
  mov_down: PointCloud with 59566 points.
FPFH dims: (33, 57450) (33, 59566)


In [67]:
# 2) Global registration (RANSAC)
result_ransac = global_registration_ransac(mov_down, ref_down, mov_fpfh, ref_fpfh, voxel_size)
print("\nRANSAC result:")
print("  fitness:", result_ransac.fitness)
print("  inlier_rmse:", result_ransac.inlier_rmse)
print("  T_ransac:\n", result_ransac.transformation)

# Visualize coarse alignment
# draw_registration_result(mov_down, ref_down, result_ransac.transformation)




RANSAC result:
  fitness: 0.7834838666353289
  inlier_rmse: 3.1019662912121815
  T_ransac:
 [[ 9.54613548e-01  2.96343564e-01 -2.98909013e-02 -6.65329423e+01]
 [-2.97143072e-01  9.40652027e-01 -1.63950475e-01 -9.35053741e+02]
 [-2.04687311e-02  1.65391218e-01  9.86015606e-01 -3.79398026e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [68]:
# 3) ICP refinement on full-resolution clouds (or you can use downsampled first)
result_icp = refine_registration_icp(mov_pcd, ref_pcd, result_ransac.transformation, voxel_size, use_point_to_plane=True)
print("\nICP result:")
print("  fitness:", result_icp.fitness)
print("  inlier_rmse:", result_icp.inlier_rmse)
print("  T_icp:\n", result_icp.transformation)

# Visualize refined alignment
#draw_registration_result(mov_pcd, ref_pcd, result_icp.transformation)




ICP result:
  fitness: 0.7907022680388273
  inlier_rmse: 1.2211446491094433
  T_icp:
 [[ 9.54329399e-01  2.97668489e-01 -2.54728901e-02 -4.74876094e+01]
 [-2.97744994e-01  9.40625151e-01 -1.63009952e-01 -9.32482540e+02]
 [-2.45624851e-02  1.63149615e-01  9.86295538e-01 -4.11525250e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [8]:
# 4) Save transformed moving point cloud (optional)
mov_aligned = copy.deepcopy(mov_pcd)
mov_aligned.transform(result_icp.transformation)
o3d.io.write_point_cloud("../Data/FPFH/mov_aligned_by_fpfh_ransac_icp_p2p.ply", mov_aligned)
print("\nSaved:", "mov_aligned_by_fpfh_ransac_icp_p2p.ply")


Saved: mov_aligned_by_fpfh_ransac_icp_p2p.ply


### Check with unity meter

In [72]:
ref_path = "Data/ICP_test/ref_Tete_D.ply"
mov_path = "Data/ICP_test/move_Dragon_01_Transform.ply"

ref_pcd = o3d.io.read_point_cloud(ref_path)
mov_pcd = o3d.io.read_point_cloud(mov_path)

print("Loaded:")
print("  ref:", ref_pcd)
print("  mov:", mov_pcd)


voxel_size = 0.005


ref_full, mov_full, ref_support, mov_support = prepare_clouds_scale_to_m(
        ref_path, mov_path, 1e-3, 0.005)

# 1) Preprocess for FPFH + RANSAC on downsampled point clouds

ref_down, ref_fpfh = preprocess_point_cloud(ref_full, voxel_size)
mov_down, mov_fpfh = preprocess_point_cloud(mov_full, voxel_size)

print("Downsampled:")
print("  ref_down:", ref_down)
print("  mov_down:", mov_down)
print("FPFH dims:", ref_fpfh.data.shape, mov_fpfh.data.shape)  # (33, N)

# 2) Global registration (RANSAC)
result_ransac = global_registration_ransac(mov_down, ref_down, mov_fpfh, ref_fpfh, voxel_size)
print("\nRANSAC result:")
print("  fitness:", result_ransac.fitness)
print("  inlier_rmse:", result_ransac.inlier_rmse)
print("  T_ransac:\n", result_ransac.transformation)

print("init_T translation:", result_ransac.transformation[:3,3])
print("ref_full diag:", np.linalg.norm(ref_full.get_axis_aligned_bounding_box().get_extent()))
print("mov_full diag:", np.linalg.norm(mov_full.get_axis_aligned_bounding_box().get_extent()))
print("ref_full center:", ref_full.get_center())
print("mov_full center:", mov_full.get_center())

# 3) ICP refinement on full-resolution clouds (or you can use downsampled first)
result_icp = refine_registration_icp(
    mov_full, ref_full,
    result_ransac.transformation,
    voxel_size,
    use_point_to_plane=True
)
print("\nICP result:")
print("  fitness:", result_icp.fitness)
print("  inlier_rmse:", result_icp.inlier_rmse)
print("  T_icp:\n", result_icp.transformation)




Loaded:
  ref: PointCloud with 1349372 points.
  mov: PointCloud with 1711170 points.
Downsampled:
  ref_down: PointCloud with 57450 points.
  mov_down: PointCloud with 59565 points.
FPFH dims: (33, 57450) (33, 59565)

RANSAC result:
  fitness: 0.7834466549147989
  inlier_rmse: 0.0028374658134148792
  T_ransac:
 [[ 0.95450323  0.29707459 -0.02588963 -0.0492163 ]
 [-0.29721961  0.94073464 -0.16333655 -0.93242842]
 [-0.02416787  0.16360017  0.98623065 -0.03930241]
 [ 0.          0.          0.          1.        ]]
init_T translation: [-0.0492163  -0.93242842 -0.03930241]
ref_full diag: 1.331735633805758
mov_full diag: 1.364668405400209
ref_full center: [-0.70876686 -0.25534312 -4.61519417]
mov_full center: [-0.71520521 -0.28435465 -4.60412256]

ICP result:
  fitness: 0.7907022680388273
  inlier_rmse: 0.0012211446536510445
  T_icp:
 [[ 0.95432939  0.29766852 -0.02547288 -0.04748758]
 [-0.29774502  0.94062514 -0.16300995 -0.93248253]
 [-0.02456249  0.16314961  0.98629554 -0.04115251]
 [ 0

### Comparison
[16:12:34] [ComputeDistances] Mean distance = 11.5045 / std deviation = 26.9021

### Evaluation 

In [38]:

def nn_distances(source: o3d.geometry.PointCloud, target: o3d.geometry.PointCloud) -> np.ndarray:
    """
    Compute nearest neighbor distances from each point in 'source' to 'target'.
    Returns:
        dists: (N,) array of Euclidean distances.
    """
    target_kd = o3d.geometry.KDTreeFlann(target)
    src_pts = np.asarray(source.points)

    dists = np.empty(len(src_pts), dtype=np.float64)
    for i, p in enumerate(src_pts):
        # 1-NN search
        _, idx, dist2 = target_kd.search_knn_vector_3d(p, 1)
        dists[i] = np.sqrt(dist2[0])
    return dists


def registration_metrics(source_aligned: o3d.geometry.PointCloud,
                         target: o3d.geometry.PointCloud,
                         thresholds=(5.0, 10.0),
                         percentiles=(50, 90, 95)) -> dict:
    """
    Compute overlap-aware metrics for registration evaluation.

    Metrics:
      - median / P90 / P95 distances
      - coverage@tau: ratio of points with dist < tau
      - trimmed_rmse@tau: RMSE computed only on points with dist < tau
      - mean / std (for reference)
    """
    d = nn_distances(source_aligned, target)

    out = {
        "mean": float(d.mean()),
        "std": float(d.std()),
        "median": float(np.percentile(d, 50)),
    }
    for p in percentiles:
        out[f"p{p}"] = float(np.percentile(d, p))

    for tau in thresholds:
        inliers = d < tau
        coverage = float(inliers.mean())
        if inliers.any():
            trimmed_rmse = float(np.sqrt(np.mean(d[inliers] ** 2)))
            trimmed_mean = float(d[inliers].mean())
        else:
            trimmed_rmse = float("nan")
            trimmed_mean = float("nan")
        out[f"coverage@{tau}"] = coverage
        out[f"trimmed_mean@{tau}"] = trimmed_mean
        out[f"trimmed_rmse@{tau}"] = trimmed_rmse

    return out


def symmetric_chamfer(source_aligned: o3d.geometry.PointCloud,
                      target: o3d.geometry.PointCloud) -> float:
    """
    Symmetric Chamfer distance (mean NN distance both directions).
    """
    d_st = nn_distances(source_aligned, target).mean()
    d_ts = nn_distances(target, source_aligned).mean()
    return float(d_st + d_ts)

In [10]:
ref_path = "../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../Data/ICP_test/move_Dragon_01_Transform.ply"
src = o3d.io.read_point_cloud(mov_path)
tgt = o3d.io.read_point_cloud(ref_path)

# src_aligned = copy.deepcopy(src)
# src_aligned.transform(T_total)

metrics = registration_metrics(src, tgt, thresholds=(5.0, 10.0))
print("Original not registered evaluation")
print(metrics)

print("Symmetric Chamfer:", symmetric_chamfer(src, tgt))

Original not registered evaluation
{'mean': 28.790589237437796, 'std': 21.611957785388448, 'median': 24.060411166186718, 'p50': 24.060411166186718, 'p90': 61.29547679266786, 'p95': 71.37153082536858, 'coverage@5.0': 0.1196292595124973, 'trimmed_mean@5.0': 2.4777668386423106, 'trimmed_rmse@5.0': 2.83796814333311, 'coverage@10.0': 0.22594189940216344, 'trimmed_mean@10.0': 4.844387508719292, 'trimmed_rmse@10.0': 5.637451842177725}
Symmetric Chamfer: 54.16179091590028


In [19]:
ref_path = "../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../Data/ICP_test/dragon_after_icp_python.ply"
src = o3d.io.read_point_cloud(mov_path)
tgt = o3d.io.read_point_cloud(ref_path)

# src_aligned = copy.deepcopy(src)
# src_aligned.transform(T_total)

metrics = registration_metrics(src, tgt, thresholds=(5.0, 10.0))
print("Manual landmark + ICP evaluation")
print(metrics)

print("Symmetric Chamfer:", symmetric_chamfer(src, tgt))

Manual landmark + ICP evaluation
{'mean': 11.900873394379946, 'std': 26.68532721206653, 'median': 0.8916812282821849, 'p50': 0.8916812282821849, 'p90': 52.385716913856236, 'p95': 79.81466731250123, 'coverage@5.0': 0.7907560324222608, 'trimmed_mean@5.0': 0.9568073840270434, 'trimmed_rmse@5.0': 1.224656404608553, 'coverage@10.0': 0.8167061133610337, 'trimmed_mean@10.0': 1.1507352380483076, 'trimmed_rmse@10.0': 1.7607851593499064}
Symmetric Chamfer: 16.668003987122248


In [12]:
ref_path = "../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../Data/FPFH/mov_aligned_by_fpfh_ransac_icp_p2p.ply"
src = o3d.io.read_point_cloud(mov_path)
tgt = o3d.io.read_point_cloud(ref_path)

# src_aligned = copy.deepcopy(src)
# src_aligned.transform(T_total)

metrics = registration_metrics(src, tgt, thresholds=(5.0, 10.0))
print("FPFH + RANSAC + ICP evaluation")
print(metrics)

print("Symmetric Chamfer:", symmetric_chamfer(src, tgt))

FPFH + RANSAC + ICP evaluation
{'mean': 11.919651183532961, 'std': 26.723596740054912, 'median': 0.8878449859671382, 'p50': 0.8878449859671382, 'p90': 52.503750839545056, 'p95': 79.93291604220614, 'coverage@5.0': 0.7907022680388273, 'trimmed_mean@5.0': 0.953115089069285, 'trimmed_rmse@5.0': 1.2211446515516156, 'coverage@10.0': 0.8165477421880936, 'trimmed_mean@10.0': 1.1466224471712876, 'trimmed_rmse@10.0': 1.7575508328904914}
Symmetric Chamfer: 16.678766929614426


### SpinNet

In [25]:
import torch
import torch.nn.functional as F
import sys, os
sys.path.insert(0, os.path.abspath("./SpinNet"))
from network.SpinNet import Descriptor_Net
from collections import OrderedDict
import time


def load_ckpt_strip_module(ckpt_path: str, map_location="cpu"):
    ckpt = torch.load(ckpt_path, map_location=map_location)

    # 情况1：直接就是 state_dict
    state_dict = ckpt

    # 情况2：ckpt 是 dict，里面有 'state_dict' 或 'model'
    if isinstance(ckpt, dict):
        if "state_dict" in ckpt:
            state_dict = ckpt["state_dict"]
        elif "model" in ckpt:
            state_dict = ckpt["model"]
        elif "net" in ckpt:
            state_dict = ckpt["net"]

    # 去掉 'module.' 前缀
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        new_k = k[7:] if k.startswith("module.") else k
        new_state_dict[new_k] = v
    return new_state_dict


def build_spinnet_model(ckpt_path,
                        des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                        voxel_r=0.04, voxel_sample=30,
                        dataset="3DMatch",
                        device="cuda:0"):

    model = Descriptor_Net(des_r, rad_n, azi_n, ele_n, voxel_r, voxel_sample, dataset)

    sd = load_ckpt_strip_module(ckpt_path, map_location="cpu")

    # strict=True 一般就能过；如果你改过网络结构才需要 strict=False
    model.load_state_dict(sd, strict=True)

    model.eval()
    model.to(device)
    return model
def spinnet_features_for_pcd(pcd_down: o3d.geometry.PointCloud,
                             model: torch.nn.Module,
                             patch_radius: float,
                             N: int = 2048,
                             batch_size: int = 64,
                             device: str = 'cuda:0'):
    """
    For each point in pcd_down, build a local patch (N,3), run SpinNet, return Open3D Feature (32, num_pts).
    """
    pts = np.asarray(pcd_down.points).astype(np.float32)
    num_pts = pts.shape[0]

    # KDTree for neighborhood query
    kdtree = o3d.geometry.KDTreeFlann(pcd_down)

    desc_list = []
    # Build patches in chunks
    patches_buf = []
    idx_buf = []

    for i in range(num_pts):
        center = pts[i]

        # radius search
        _, idxs, _ = kdtree.search_radius_vector_3d(center, patch_radius)
        if len(idxs) < 5:
            # fallback: if neighborhood is too small, just repeat center
            patch = np.repeat(center[None, :], N, axis=0)
        else:
            neigh = pts[np.asarray(idxs, dtype=np.int64)]

            # sample to fixed N
            if neigh.shape[0] >= N:
                sel = np.random.choice(neigh.shape[0], N, replace=False)
                patch = neigh[sel]
            else:
                # pad by resampling with replacement
                sel = np.random.choice(neigh.shape[0], N, replace=True)
                patch = neigh[sel]

            # IMPORTANT: ensure the last point is the center (SpinNet uses input[:, -1, :] as center)
            patch[-1] = center

        patches_buf.append(patch)
        idx_buf.append(i)

        # run a batch
        if len(patches_buf) == batch_size or i == num_pts - 1:
            batch = torch.from_numpy(np.stack(patches_buf, axis=0)).to(device=device, dtype=torch.float32)  # (B,N,3)
            with torch.no_grad():
                out = model(batch)                   # (B,32,1,1)
                out = out.view(out.shape[0], -1)     # (B,32)
                out = F.normalize(out, p=2, dim=1)   # (B,32)
            desc_list.append(out.detach().cpu().numpy())
            patches_buf = []
            idx_buf = []

    desc = np.concatenate(desc_list, axis=0)  # (num_pts, 32)

    feat = o3d.pipelines.registration.Feature()
    feat.data = desc.T  # Open3D expects (dim, num_pts)
    return feat
    
def neighborhood_stats(pcd_down, radius, max_check=5000):
    pts = np.asarray(pcd_down.points)
    kdtree = o3d.geometry.KDTreeFlann(pcd_down)
    n = pts.shape[0]
    idxs = np.random.choice(n, size=min(n, max_check), replace=False)

    counts = []
    t0 = time.time()
    for i in idxs:
        _, nn, _ = kdtree.search_radius_vector_3d(pts[i], radius)
        counts.append(len(nn))
    t = time.time() - t0
    counts = np.array(counts)
    print(f"[radius={radius}] checked {len(idxs)} points in {t:.2f}s")
    print("  nn count: min / median / p90 / max =",
          counts.min(), np.median(counts), np.percentile(counts, 90), counts.max())
    return counts

def spinnet_features_for_pcd_profiled(
    pcd_down: o3d.geometry.PointCloud,
    model: torch.nn.Module,
    patch_radius: float,
    N: int = 2048,
    batch_size: int = 64,
    device: str = "cuda:0",
    warmup_batches: int = 2,
):
    pts = np.asarray(pcd_down.points).astype(np.float32)
    num_pts = pts.shape[0]
    kdtree = o3d.geometry.KDTreeFlann(pcd_down)

    # timers
    t_kdtree = 0.0
    t_patch  = 0.0
    t_stack  = 0.0
    t_fwd    = 0.0

    desc_list = []
    patches_buf = []

    # optional warmup (use first few points)
    if "cuda" in device and torch.cuda.is_available():
        for _ in range(warmup_batches):
            # build a tiny dummy batch
            B = min(batch_size, max(1, num_pts))
            dummy = torch.randn(B, N, 3, device=device, dtype=torch.float32)
            with torch.no_grad():
                torch.cuda.synchronize()
                _ = model(dummy)
                torch.cuda.synchronize()

    t_total0 = time.perf_counter()

    for i in range(num_pts):
        center = pts[i]

        # 1) KDTree radius search timing
        t0 = time.perf_counter()
        _, idxs, _ = kdtree.search_radius_vector_3d(center, patch_radius)
        t_kdtree += time.perf_counter() - t0

        # 2) Patch build timing (sampling/padding)
        t0 = time.perf_counter()
        if len(idxs) < 5:
            patch = np.repeat(center[None, :], N, axis=0)
        else:
            neigh = pts[np.asarray(idxs, dtype=np.int64)]
            if neigh.shape[0] >= N:
                sel = np.random.choice(neigh.shape[0], N, replace=False)
            else:
                sel = np.random.choice(neigh.shape[0], N, replace=True)
            patch = neigh[sel]
            patch[-1] = center
        t_patch += time.perf_counter() - t0

        patches_buf.append(patch)

        # 3) batch forward timing
        if len(patches_buf) == batch_size or i == num_pts - 1:
            t0 = time.perf_counter()
            batch_np = np.stack(patches_buf, axis=0)   # (B,N,3)
            t_stack += time.perf_counter() - t0

            batch = torch.from_numpy(batch_np).to(device=device, dtype=torch.float32)

            if "cuda" in device and torch.cuda.is_available():
                torch.cuda.synchronize()
            t0 = time.perf_counter()
            with torch.no_grad():
                out = model(batch)               # (B,32,1,1) or similar
                out = out.view(out.shape[0], -1) # (B,32)
                out = F.normalize(out, p=2, dim=1)
            if "cuda" in device and torch.cuda.is_available():
                torch.cuda.synchronize()
            t_fwd += time.perf_counter() - t0

            desc_list.append(out.detach().cpu().numpy())
            patches_buf = []

    t_total = time.perf_counter() - t_total0

    desc = np.concatenate(desc_list, axis=0)  # (num_pts, 32)
    feat = o3d.pipelines.registration.Feature()
    feat.data = desc.T  # (32, num_pts)

    profile = {
        "num_pts": num_pts,
        "total_s": t_total,
        "per_point_ms": (t_total / max(1, num_pts)) * 1000.0,
        "kdtree_s": t_kdtree,
        "patch_build_s": t_patch,
        "stack_s": t_stack,
        "forward_s": t_fwd,
        "kdtree_ms_per_pt": (t_kdtree / max(1, num_pts)) * 1000.0,
        "patch_ms_per_pt": (t_patch / max(1, num_pts)) * 1000.0,
        "stack_ms_per_pt": (t_stack / max(1, num_pts)) * 1000.0,
        "fwd_ms_per_pt": (t_fwd / max(1, num_pts)) * 1000.0,
    }
    return feat, profile


In [11]:
ref_path = "Data/ICP_test/ref_Tete_D.ply"
mov_path = "Data/ICP_test/move_Dragon_01_Transform.ply"

ref_pcd = o3d.io.read_point_cloud(ref_path)
mov_pcd = o3d.io.read_point_cloud(mov_path)


bbox = ref_pcd.get_axis_aligned_bounding_box()
diag = np.linalg.norm(bbox.get_extent())
print("diag =", diag) # Verify the len unity

# 0) Unity transfer
scale = 1e-3
ref_pcd.scale(scale, center=ref_pcd.get_center())
mov_pcd.scale(scale, center=mov_pcd.get_center())

# 1) downsample（unity：m）
voxel_size = 0.005  # 5mm = 0.005m
ref_down = ref_pcd.voxel_down_sample(voxel_size)
mov_down = mov_pcd.voxel_down_sample(voxel_size)

ref_down, _ = ref_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
mov_down, _ = mov_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

bbox = ref_pcd.get_axis_aligned_bounding_box()
diag = np.linalg.norm(bbox.get_extent())
print("After scale, diag =", diag) # Verify the len unity

print("ref_down points:", np.asarray(ref_down.points).shape[0])
print("mov_down points:", np.asarray(mov_down.points).shape[0])

diag = 1331.7356338057582
After scale, diag = 1.3317356338051947


In [15]:
# 2) Build SpinNet model
ckpt_path = "SpinNet/pre-trained_models/3DMatch_best.pkl"   
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda:0")

/tmp/ipykernel_4027753/2108995958.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=map_location)


In [24]:
# 2.5)
print("Loaded ")
print("  ref_down:", ref_down)
print("  mov_down:", mov_down)

neighborhood_stats(ref_down, radius=0.30)

Loaded 
  ref_down: PointCloud with 57451 points.
  mov_down: PointCloud with 59565 points.
[radius=0.3] checked 5000 points in 10.37s
  nn count: min / median / p90 / max = 4800 24746.5 32293.500000000007 36282


array([32898, 23067, 28527, ..., 32842, 23483, 26997])

In [27]:
# 2.5) Time test
idx = np.random.choice(len(ref_down.points), 2000, replace=False)
ref_sub = ref_down.select_by_index(idx)

_, prof_sub = spinnet_features_for_pcd_profiled(
    ref_sub, model, patch_radius=0.30, N=2048, batch_size=32, device="cuda:0"
)

est_total = prof_sub["per_point_ms"] * len(ref_down.points) / 1000.0
print("estimated total seconds on full ref_down:", est_total)

estimated total seconds on full ref_down: 212.8129515532977


In [28]:
print(prof_sub)

{'num_pts': 2000, 'total_s': 7.408502952195704, 'per_point_ms': 3.704251476097852, 'kdtree_s': 0.1374448137357831, 'patch_build_s': 0.21178276371210814, 'stack_s': 0.019267170690000057, 'forward_s': 7.004434240050614, 'kdtree_ms_per_pt': 0.06872240686789155, 'patch_ms_per_pt': 0.10589138185605407, 'stack_ms_per_pt': 0.009633585345000029, 'fwd_ms_per_pt': 3.502217120025307}


In [30]:
# 3) SpinNet feature（patch_radius：m）
ref_feat, ref_prof = spinnet_features_for_pcd_profiled(ref_down, model, patch_radius=0.30, N=2048, batch_size=48, device="cuda:0")


#ref_feat = spinnet_features_for_pcd(ref_down, model, patch_radius=0.30, N=2048, batch_size=48, device="cuda:0")
#mov_feat = spinnet_features_for_pcd(mov_down, model, patch_radius=0.30, N=2048, batch_size=48, device="cuda:0")



In [32]:
print(ref_prof)

{'num_pts': 57451, 'total_s': 366.79874674696475, 'per_point_ms': 6.3845493855105175, 'kdtree_s': 116.37929208111018, 'patch_build_s': 55.9014504281804, 'stack_s': 0.3908678153529763, 'forward_s': 193.26412159670144, 'kdtree_ms_per_pt': 2.0257139489497167, 'patch_ms_per_pt': 0.9730283272385232, 'stack_ms_per_pt': 0.00680349890085423, 'fwd_ms_per_pt': 3.363981855784955}


In [35]:
mov_feat, mov_prof = spinnet_features_for_pcd_profiled(mov_down, model, patch_radius=0.30, N=2048, batch_size=48, device="cuda:0")

In [36]:
print(mov_prof)

{'num_pts': 59565, 'total_s': 353.89798430912197, 'per_point_ms': 5.941374705097322, 'kdtree_s': 107.84113709721714, 'patch_build_s': 52.47581277042627, 'stack_s': 0.3811010494828224, 'forward_s': 192.4380912426859, 'kdtree_ms_per_pt': 1.8104782522826683, 'patch_ms_per_pt': 0.8809840136057462, 'stack_ms_per_pt': 0.006398070166756021, 'fwd_ms_per_pt': 3.23072427168112}


In [40]:
# 4) RANSAC 
result_ransac = global_registration_ransac(mov_down, ref_down, mov_feat, ref_feat, voxel_size)
print("\nRANSAC result:")
print("  fitness:", result_ransac.fitness)
print("  inlier_rmse:", result_ransac.inlier_rmse)
print("  T_ransac:\n", result_ransac.transformation)



[Open3D WARNING] Too few correspondences (514) after mutual filter, fall back to original correspondences.

RANSAC result:
  fitness: 0.6604213883992277
  inlier_rmse: 0.004330643965148919
  T_ransac:
 [[ 9.43375853e-01  3.31017676e-01 -2.16632845e-02 -3.96642037e+01]
 [-3.29924565e-01  9.29454455e-01 -1.65118739e-01 -9.87221529e+02]
 [-3.45221849e-02  1.62916281e-01  9.86035752e-01 -5.37294406e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [41]:
# 5) ICP
result_icp = refine_registration_icp(mov_pcd, ref_pcd, result_ransac.transformation, voxel_size, use_point_to_plane=False)
print("\nICP result:")
print("  fitness:", result_icp.fitness)
print("  inlier_rmse:", result_icp.inlier_rmse)
print("  T_icp:\n", result_icp.transformation)



ICP result:
  fitness: 0.7907040212252435
  inlier_rmse: 0.0012213668000526324
  T_icp:
 [[ 9.54331532e-01  2.97661486e-01 -2.54748046e-02 -5.88602838e+01]
 [-2.97738628e-01  9.40628721e-01 -1.63000974e-01 -9.51272785e+02]
 [-2.45567794e-02  1.63141803e-01  9.86296972e-01 -4.53309096e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [42]:
# 4) Save transformed moving point cloud (optional)
mov_aligned = copy.deepcopy(mov_pcd)
mov_aligned.transform(result_icp.transformation)
o3d.io.write_point_cloud("Data/SpinNet/mov_aligned_by_SpinNet_ransac_icp_p2p.ply", mov_aligned)
print("\nSaved:", "mov_aligned_by_SpinNet_ransac_icp_p2p.ply")


Saved: mov_aligned_by_SpinNet_ransac_icp_p2p.ply


In [50]:
def preprocess_for_registration(pcd, scale=1e-3):
    p = o3d.geometry.PointCloud(pcd)  
    c = p.get_center()
    p.scale(scale, center=c)
    return p

In [51]:
ref_path = "Data/ICP_test/ref_Tete_D.ply"
mov_path = "Data/SpinNet/mov_aligned_by_SpinNet_ransac_icp_p2p.ply"


src = o3d.io.read_point_cloud(mov_path)
tgt = o3d.io.read_point_cloud(ref_path)

tgt = preprocess_for_registration(tgt)

metrics = registration_metrics(src, tgt, thresholds=(0.005, 0.010))
print(metrics)
print("Symmetric Chamfer:", symmetric_chamfer(src, tgt))


{'mean': 0.011918822987517803, 'std': 0.026722915787976767, 'median': 0.0008861635487700434, 'p50': 0.0008861635487700434, 'p90': 0.05249955660834527, 'p95': 0.07992664360168701, 'coverage@0.005': 0.7907040212252435, 'trimmed_mean@0.005': 0.0009527310011387557, 'trimmed_rmse@0.005': 0.0012213667999690684, 'coverage@0.01': 0.8165454046062051, 'trimmed_mean@0.01': 0.0011462205703362967, 'trimmed_rmse@0.01': 0.0017576162087935656}
Symmetric Chamfer: 0.016677589187423562


### Training SpinNet
load training files ../../data/3DMatch/patches/train/train_anc&pos_20_2048_2000.pkl
Epoch 19: Loss 1.0435524266898795, time 10648.4401s
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_4000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_6000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_8000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_10000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_12000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_14000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_16000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_17043.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_2000.pkl
Evaluation: Epoch 19: Loss 1.2269546277540968
Avg one epoch time: 10514.17, total 20 epochs time: 214677.52
Training finish!... save training results

### Compare FPFH + ICP vs SpinNet + ICP on different downsample keypoints 

In [76]:
import json


def now():
    return time.perf_counter()

def diag(pcd: o3d.geometry.PointCloud) -> float:
    return float(np.linalg.norm(pcd.get_axis_aligned_bounding_box().get_extent()))

def select_keypoints_random(pcd: o3d.geometry.PointCloud, K: int, seed: int = 0):
    pts = np.asarray(pcd.points)
    n = pts.shape[0]
    if K >= n:
        idx = np.arange(n, dtype=np.int64)
    else:
        rng = np.random.default_rng(seed)
        idx = rng.choice(n, size=K, replace=False).astype(np.int64)

    idx.sort()  # align with o3d order
    kp = pcd.select_by_index(idx.tolist())
    return kp, idx



def compute_fpfh_for_support(pcd_support: o3d.geometry.PointCloud, voxel_size: float):
    """
    Compute FPFH on a SUPPORT cloud (already downsampled + outlier-removed).
    Returns:
        fpfh: o3d.pipelines.registration.Feature (33, N_support)
    """
    # Normals
    radius_normal = voxel_size * 2.0
    pcd_support.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
    )

    # FPFH
    radius_feature = voxel_size * 5.0
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_support,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100)
    )
    return fpfh

def slice_feature_by_index(feat: o3d.pipelines.registration.Feature, idx: np.ndarray):
    """
    feat.data shape: (dim, N). Returns a Feature with shape (dim, K).
    """
    f = o3d.pipelines.registration.Feature()
    f.data = feat.data[:, idx]
    return f


def spinnet_features_for_keypoints_profiled(
    support_pcd: o3d.geometry.PointCloud,   # KDTree & neighbor source
    query_pcd: o3d.geometry.PointCloud,     # centers to compute descriptors (K points)
    model: torch.nn.Module,
    patch_radius: float,
    N: int = 2048,
    batch_size: int = 64,
    device: str = "cuda:0",
    warmup_batches: int = 2,
):
    support_pts = np.asarray(support_pcd.points).astype(np.float32)
    query_pts   = np.asarray(query_pcd.points).astype(np.float32)
    num_q = query_pts.shape[0]

    kdtree = o3d.geometry.KDTreeFlann(support_pcd)

    # timers
    t_kdtree = 0.0
    t_patch  = 0.0
    t_stack  = 0.0
    t_fwd    = 0.0

    desc_list = []
    patches_buf = []

    # Model sanity check
    if "cuda" in device and torch.cuda.is_available():
        for _ in range(warmup_batches):
            B = min(batch_size, max(1, num_q))
            dummy = torch.randn(B, N, 3, device=device, dtype=torch.float32)
            with torch.no_grad():
                torch.cuda.synchronize()
                _ = model(dummy)
                torch.cuda.synchronize()

    t_total0 = time.perf_counter()

    for i in range(num_q):
        center = query_pts[i]

        # 1) KDTree radius search on SUPPORT
        t0 = time.perf_counter()
        _, idxs, _ = kdtree.search_radius_vector_3d(center, patch_radius)
        t_kdtree += time.perf_counter() - t0

        # 2) Patch build from SUPPORT points, but force last point = QUERY center
        t0 = time.perf_counter()
        if len(idxs) < 5:
            patch = np.repeat(center[None, :], N, axis=0)
        else:
            neigh = support_pts[np.asarray(idxs, dtype=np.int64)]
            if neigh.shape[0] >= N:
                sel = np.random.choice(neigh.shape[0], N, replace=False)
            else:
                sel = np.random.choice(neigh.shape[0], N, replace=True)
            patch = neigh[sel]
            patch[-1] = center
        t_patch += time.perf_counter() - t0

        patches_buf.append(patch)

        # 3) batch forward
        if len(patches_buf) == batch_size or i == num_q - 1:
            t0 = time.perf_counter()
            batch_np = np.stack(patches_buf, axis=0)
            t_stack += time.perf_counter() - t0

            batch = torch.from_numpy(batch_np).to(device=device, dtype=torch.float32)

            if "cuda" in device and torch.cuda.is_available():
                torch.cuda.synchronize()
            t0 = time.perf_counter()
            with torch.no_grad():
                out = model(batch)
                out = out.view(out.shape[0], -1)
                out = F.normalize(out, p=2, dim=1)
            if "cuda" in device and torch.cuda.is_available():
                torch.cuda.synchronize()
            t_fwd += time.perf_counter() - t0

            desc_list.append(out.detach().cpu().numpy())
            patches_buf = []

    t_total = time.perf_counter() - t_total0

    desc = np.concatenate(desc_list, axis=0)  # (K, 32)
    feat = o3d.pipelines.registration.Feature()
    feat.data = desc.T  # (32, K)

    profile = {
        "num_query": int(num_q),
        "total_s": float(t_total),
        "per_query_ms": float((t_total / max(1, num_q)) * 1000.0),
        "kdtree_s": float(t_kdtree),
        "patch_build_s": float(t_patch),
        "stack_s": float(t_stack),
        "forward_s": float(t_fwd),
        "kdtree_ms_per_q": float((t_kdtree / max(1, num_q)) * 1000.0),
        "patch_ms_per_q": float((t_patch / max(1, num_q)) * 1000.0),
        "stack_ms_per_q": float((t_stack / max(1, num_q)) * 1000.0),
        "fwd_ms_per_q": float((t_fwd / max(1, num_q)) * 1000.0),
    }
    return feat, profile

def prepare_clouds_scale_to_m(
    ref_path: str,
    mov_path: str,
    scale_to_m: float = 1e-3,
    voxel_size: float = 0.005,
    outlier_nb: int = 20,
    outlier_std: float = 2.0,
):
    ref_raw = o3d.io.read_point_cloud(ref_path)
    mov_raw = o3d.io.read_point_cloud(mov_path)

    ref_full = copy.deepcopy(ref_raw)
    mov_full = copy.deepcopy(mov_raw)

    # TRUE unit conversion: scale around origin
    ref_full.scale(scale_to_m, center=(0.0, 0.0, 0.0))
    mov_full.scale(scale_to_m, center=(0.0, 0.0, 0.0))

    # support clouds in meters
    ref_support = ref_full.voxel_down_sample(voxel_size)
    mov_support = mov_full.voxel_down_sample(voxel_size)

    ref_support, _ = ref_support.remove_statistical_outlier(nb_neighbors=outlier_nb, std_ratio=outlier_std)
    mov_support, _ = mov_support.remove_statistical_outlier(nb_neighbors=outlier_nb, std_ratio=outlier_std)

    return ref_full, mov_full, ref_support, mov_support

def save_aligned_cloud(
    pcd: o3d.geometry.PointCloud,
    base_dir: str,
    K: int,
    seed: int,
    method: str,
):
    """
    Save aligned point cloud to:
    base_dir/K_<K>/seed_<seed>/mov_aligned_<method>.ply
    """
    save_dir = os.path.join(base_dir, f"K_{K}", f"seed_{seed}")
    os.makedirs(save_dir, exist_ok=True)

    filename = f"mov_aligned_{method}.ply"
    path = os.path.join(save_dir, filename)

    o3d.io.write_point_cloud(path, pcd)
    print(f"[Saved] {path}")

    return path

def assert_kp_alignment(support_pcd, kp_pcd, idx, name, tol=1e-12, n_check=20):
    sup = np.asarray(support_pcd.points)
    kp  = np.asarray(kp_pcd.points)
    idx = np.asarray(idx, dtype=np.int64)

    assert kp.shape[0] == idx.shape[0], f"{name}: kp_n != idx_n"
    m = min(n_check, kp.shape[0])
    for j in range(m):
        d = np.linalg.norm(kp[j] - sup[idx[j]])
        assert d < tol, f"{name}: order mismatch at j={j}, ||kp-sup[idx]||={d}"



def run_one_K_seed_compare_fpfh_spinnet(
    ref_full: o3d.geometry.PointCloud,
    mov_full: o3d.geometry.PointCloud,
    ref_support: o3d.geometry.PointCloud,
    mov_support: o3d.geometry.PointCloud,
    model: torch.nn.Module,
    voxel_size: float,
    patch_radius: float,
    K: int,
    seed: int,
    N_patch: int = 2048,
    batch_size: int = 48,
    device: str = "cuda:0",
    use_point_to_plane: bool = True,
    eval_thresholds_m=(0.005, 0.010),
):
    """
    Returns a dict with:
      - keypoint info
      - timing breakdown for both methods
      - RANSAC/ICP metrics and your evaluation metrics
    """
    out = {
        "K": int(K),
        "seed": int(seed),
        "voxel_size": float(voxel_size),
        "patch_radius": float(patch_radius),
    }

    # --- Keypoints, same idx for both methods ---
    ref_kp, idx_ref = select_keypoints_random(ref_support, K, seed=seed)
    mov_kp, idx_mov = select_keypoints_random(mov_support, K, seed=seed)

    out["ref_support_n"] = int(len(ref_support.points))
    out["mov_support_n"] = int(len(mov_support.points))
    out["ref_kp_n"] = int(len(ref_kp.points))
    out["mov_kp_n"] = int(len(mov_kp.points))

    # ============================================================
    # Method 1: FPFH (support full -> slice K)
    # ============================================================
    t0 = now()
    ref_fpfh_all = compute_fpfh_for_support(ref_support, voxel_size)
    mov_fpfh_all = compute_fpfh_for_support(mov_support, voxel_size)
    t_fpfh_feat = now() - t0

    ref_fpfh_kp = slice_feature_by_index(ref_fpfh_all, idx_ref)
    mov_fpfh_kp = slice_feature_by_index(mov_fpfh_all, idx_mov)
    
    assert_kp_alignment(ref_support, ref_kp, idx_ref, "ref")
    assert_kp_alignment(mov_support, mov_kp, idx_mov, "mov")
    # RANSAC
    t0 = now()
    ransac_fpfh = global_registration_ransac(mov_kp, ref_kp, mov_fpfh_kp, ref_fpfh_kp, voxel_size)
    t_fpfh_ransac = now() - t0

    # ICP
    t0 = now()
    icp_fpfh = refine_registration_icp(mov_full, ref_full, ransac_fpfh.transformation, voxel_size, use_point_to_plane=use_point_to_plane)
    t_fpfh_icp = now() - t0

    # Evaluate
    mov_aligned_fpfh = copy.deepcopy(mov_full)
    mov_aligned_fpfh.transform(icp_fpfh.transformation)

    eval_fpfh = registration_metrics(mov_aligned_fpfh, ref_full, thresholds=eval_thresholds_m)
    chamfer_fpfh = symmetric_chamfer(mov_aligned_fpfh, ref_full)

    out.update({
        "fpfh_feat_s": float(t_fpfh_feat),
        "fpfh_ransac_s": float(t_fpfh_ransac),
        "fpfh_icp_s": float(t_fpfh_icp),
        "fpfh_total_s": float(t_fpfh_feat + t_fpfh_ransac + t_fpfh_icp),

        "fpfh_ransac_fitness": float(ransac_fpfh.fitness),
        "fpfh_ransac_rmse": float(ransac_fpfh.inlier_rmse),
        "fpfh_icp_fitness": float(icp_fpfh.fitness),
        "fpfh_icp_rmse": float(icp_fpfh.inlier_rmse),

        "fpfh_cov_5mm": float(eval_fpfh.get(f"coverage@{eval_thresholds_m[0]}", np.nan)),
        "fpfh_cov_10mm": float(eval_fpfh.get(f"coverage@{eval_thresholds_m[1]}", np.nan)),
        "fpfh_trimrmse_5mm": float(eval_fpfh.get(f"trimmed_rmse@{eval_thresholds_m[0]}", np.nan)),
        "fpfh_trimrmse_10mm": float(eval_fpfh.get(f"trimmed_rmse@{eval_thresholds_m[1]}", np.nan)),
        "fpfh_chamfer": float(chamfer_fpfh),
    })

    # ============================================================
    # Method 2: SpinNet (support/query separated)
    # ============================================================
    # Feature extraction (profile already contains time)
    sp_ref_feat, sp_ref_prof = spinnet_features_for_keypoints_profiled(
        support_pcd=ref_support,
        query_pcd=ref_kp,
        model=model,
        patch_radius=patch_radius,
        N=N_patch,
        batch_size=batch_size,
        device=device,
    )
    sp_mov_feat, sp_mov_prof = spinnet_features_for_keypoints_profiled(
        support_pcd=mov_support,
        query_pcd=mov_kp,
        model=model,
        patch_radius=patch_radius,
        N=N_patch,
        batch_size=batch_size,
        device=device,
    )

    t_sp_feat = sp_ref_prof["total_s"] + sp_mov_prof["total_s"]

    # RANSAC
    t0 = now()
    ransac_sp = global_registration_ransac(mov_kp, ref_kp, sp_mov_feat, sp_ref_feat, voxel_size)
    t_sp_ransac = now() - t0

    # ICP
    t0 = now()
    icp_sp = refine_registration_icp(mov_full, ref_full, ransac_sp.transformation, voxel_size, use_point_to_plane=use_point_to_plane)
    t_sp_icp = now() - t0

    # Evaluate
    mov_aligned_sp = copy.deepcopy(mov_full)
    mov_aligned_sp.transform(icp_sp.transformation)

    eval_sp = registration_metrics(mov_aligned_sp, ref_full, thresholds=eval_thresholds_m)
    chamfer_sp = symmetric_chamfer(mov_aligned_sp, ref_full)

    out.update({
        "spinnet_ref_feat_s": float(sp_ref_prof["total_s"]),
        "spinnet_mov_feat_s": float(sp_mov_prof["total_s"]),
        "spinnet_feat_s": float(t_sp_feat),
        "spinnet_ransac_s": float(t_sp_ransac),
        "spinnet_icp_s": float(t_sp_icp),
        "spinnet_total_s": float(t_sp_feat + t_sp_ransac + t_sp_icp),

        "spinnet_ransac_fitness": float(ransac_sp.fitness),
        "spinnet_ransac_rmse": float(ransac_sp.inlier_rmse),
        "spinnet_icp_fitness": float(icp_sp.fitness),
        "spinnet_icp_rmse": float(icp_sp.inlier_rmse),

        "spinnet_cov_5mm": float(eval_sp.get(f"coverage@{eval_thresholds_m[0]}", np.nan)),
        "spinnet_cov_10mm": float(eval_sp.get(f"coverage@{eval_thresholds_m[1]}", np.nan)),
        "spinnet_trimrmse_5mm": float(eval_sp.get(f"trimmed_rmse@{eval_thresholds_m[0]}", np.nan)),
        "spinnet_trimrmse_10mm": float(eval_sp.get(f"trimmed_rmse@{eval_thresholds_m[1]}", np.nan)),
        "spinnet_chamfer": float(chamfer_sp),
    })

    out["fpfh_T_icp"] = icp_fpfh.transformation
    out["spinnet_T_icp"] = icp_sp.transformation


    # Save all results:
    save_base = "Data/Registrations"

    save_aligned_cloud(
    pcd=mov_aligned_fpfh,
    base_dir=save_base,
    K=K,
    seed=seed,
    method="FPFH",
    )

    save_aligned_cloud(
    pcd=mov_aligned_sp,
    base_dir=save_base,
    K=K,
    seed=seed,
    method="SpinNet",
    )
    return out


def save_out_json(out: dict, base_dir: str, K: int, seed: int):
    """
    Save the full `out` dict to:
    base_dir/K_<K>/seed_<seed>/out.json
    """
    save_dir = os.path.join(base_dir, f"K_{K}", f"seed_{seed}")
    os.makedirs(save_dir, exist_ok=True)

    path = os.path.join(save_dir, "out.json")

    # numpy -> python float/int 
    def convert(o):
        if hasattr(o, "tolist"):
            return o.tolist()
        return o

    with open(path, "w") as f:
        json.dump(out, f, indent=2, default=convert)

    print(f"[Saved] {path}")
    return path


In [77]:
def run_sweep(
    ref_path: str,
    mov_path: str,
    model: torch.nn.Module,
    Ks=(512, 1024, 2048, 4096, 8192, 16384, 32768),
    seeds=(0, 1, 2),
    scale_to_m: float = 1e-3,
    voxel_size: float = 0.005,
    patch_radius: float = 0.30,
    N_patch: int = 2048,
    batch_size: int = 96,
    device: str = "cuda:0",
    use_point_to_plane: bool = True,
    eval_thresholds_m=(0.005, 0.010),
):
    # Prepare clouds once (shared by all runs)
    ref_full, mov_full, ref_support, mov_support = prepare_clouds_scale_to_m(
        ref_path, mov_path, scale_to_m=scale_to_m, voxel_size=voxel_size
    )

    print("Prepared:")
    print("  ref_full diag:", diag(ref_full), "center:", ref_full.get_center())
    print("  mov_full diag:", diag(mov_full), "center:", mov_full.get_center())
    print("  ref_support n:", len(ref_support.points))
    print("  mov_support n:", len(mov_support.points))

    results = []
    for K in Ks:
        for seed in seeds:
            print(f"\n=== Run K={K}, seed={seed} ===")
            out = run_one_K_seed_compare_fpfh_spinnet(
                ref_full=ref_full,
                mov_full=mov_full,
                ref_support=ref_support,
                mov_support=mov_support,
                model=model,
                voxel_size=voxel_size,
                patch_radius=patch_radius,
                K=K,
                seed=seed,
                N_patch=N_patch,
                batch_size=batch_size,
                device=device,
                use_point_to_plane=use_point_to_plane,
                eval_thresholds_m=eval_thresholds_m,
            )
            save_out_json(out=out, base_dir="Data/Registrations", K=K, seed=seed,)
            results.append(out)
            print("  FPFH total_s:", out["fpfh_total_s"], "cov@5mm:", out["fpfh_cov_5mm"], "rmse@5mm:", out["fpfh_trimrmse_5mm"])
            print("  Spin total_s:", out["spinnet_total_s"], "cov@5mm:", out["spinnet_cov_5mm"], "rmse@5mm:", out["spinnet_trimrmse_5mm"])

    return results

In [78]:
ref_path = "Data/ICP_test/ref_Tete_D.ply"
mov_path = "Data/ICP_test/move_Dragon_01_Transform.ply"
results = run_sweep(ref_path, mov_path, model)

Prepared:
  ref_full diag: 1.331735633805758 center: [-0.70876686 -0.25534312 -4.61519417]
  mov_full diag: 1.364668405400209 center: [-0.71520521 -0.28435465 -4.60412256]
  ref_support n: 57450
  mov_support n: 59565

=== Run K=512, seed=0 ===
[Saved] Data/Registrations/K_512/seed_0/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_512/seed_0/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/K_512/seed_0/out.json
  FPFH total_s: 7.6443045837804675 cov@5mm: 0.7907016836433551 rmse@5mm: 0.0012211375524111196
  Spin total_s: 12.858805415220559 cov@5mm: 0.7907016836433551 rmse@5mm: 0.00122113757617147

=== Run K=512, seed=1 ===
[Saved] Data/Registrations/K_512/seed_1/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_512/seed_1/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/K_512/seed_1/out.json
  FPFH total_s: 31.15536557789892 cov@5mm: 0.14580199512614175 rmse@5mm: 0.002839405135798507
  Spin total_s: 15.11713317129761 cov@5mm: 0.7907022680388273 rmse@5mm: 0.0012211446531864662

===

In [79]:
print(results)

[{'K': 512, 'seed': 0, 'voxel_size': 0.005, 'patch_radius': 0.3, 'ref_support_n': 57450, 'mov_support_n': 59565, 'ref_kp_n': 512, 'mov_kp_n': 512, 'fpfh_feat_s': 0.3970149103552103, 'fpfh_ransac_s': 0.1712689409032464, 'fpfh_icp_s': 7.076020732522011, 'fpfh_total_s': 7.6443045837804675, 'fpfh_ransac_fitness': 0.048828125, 'fpfh_ransac_rmse': 0.005740113929802985, 'fpfh_icp_fitness': 0.7907016836433551, 'fpfh_icp_rmse': 0.0012211375524111333, 'fpfh_cov_5mm': 0.7907016836433551, 'fpfh_cov_10mm': 0.8165477421880936, 'fpfh_trimrmse_5mm': 0.0012211375524111196, 'fpfh_trimrmse_10mm': 0.001757551096756306, 'fpfh_chamfer': 0.0166787665260775, 'spinnet_ref_feat_s': 3.0425073839724064, 'spinnet_mov_feat_s': 3.0006922790780663, 'spinnet_feat_s': 6.043199663050473, 'spinnet_ransac_s': 0.17391147557646036, 'spinnet_icp_s': 6.6416942765936255, 'spinnet_total_s': 12.858805415220559, 'spinnet_ransac_fitness': 0.052734375, 'spinnet_ransac_rmse': 0.005421764131669186, 'spinnet_icp_fitness': 0.7907016836